In [1]:
import nltk
import scipy
import pandas as pd
import numpy as np

import csv
from nltk.corpus import wordnet as wn
from libs import *

In [2]:
def main():
    word_sym=pd.read_csv('data\\WordSim353.csv')
    word_sym.insert(3,"wu_palmer",np.zeros(353))
    word_sym.insert(4,"shortest_path",np.zeros(353))
    word_sym.insert(5,"leak_chod",np.zeros(353))
    #print(word_sym)
    for i in range(0,353):
        v1 = word_sym.at[i,'Word 1']
        v2 = word_sym.at[i,'Word 2']
        max_wu=0
        max_sp=0
        max_lch=0
        for s1 in wn.synsets(v1):
            for s2 in wn.synsets(v2):
                actual_wu=wu_similarity(s1,s2)
                actual_sp=wn.wup_similarity(s1,s2)
                actual_lch=1
                if actual_wu>max_wu:
                    max_wu=actual_wu
                if actual_sp>max_sp:
                    max_sp=actual_sp
                if actual_lch>max_lch:
                    max_lch=actual_lch
        word_sym.iat[i,3]= max_wu
        word_sym.iat[i,4]= max_sp
        word_sym.iat[i,5]= max_lch

    print(word_sym.to_string())


if __name__ == "__main__":
    main()



            Word 1          Word 2  Human (mean)  wu_palmer  shortest_path  leak_chod
0             love             sex          6.77      0.923       0.923077        1.0
1            tiger             cat          7.35      0.966       0.965517        1.0
2            tiger           tiger         10.00      1.000       1.000000        1.0
3             book           paper          7.46      0.875       0.875000        1.0
4         computer        keyboard          7.62      0.824       0.823529        1.0
5         computer        internet          7.58      0.632       0.631579        1.0
6            plane             car          5.77      0.762       0.727273        1.0
7            train             car          6.31      0.778       0.736842        1.0
8        telephone   communication          7.50      0.167       0.200000        1.0
9       television           radio          6.77      0.909       0.909091        1.0
10           media           radio          7.42      